In [1]:
# Okay, in this work, we will still not be storing the extracted features that we will be using in the stage 2 model, but rather focus on the model performnace, incase we need to report this somewhere.

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from data_generator import DataGenerator, batch_predict

2025-08-28 11:35:53.361851: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-28 11:35:53.477957: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-28 11:35:53.477984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-28 11:35:53.478910: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-28 11:35:53.484816: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# import the model here?

In [4]:
model = tf.keras.models.load_model("models/CNN_seq2seq_overlapping.keras")

2025-08-28 11:36:30.715767: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [5]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 13, None, None, 3)   0         []                            
                             ]                                                                    
                                                                                                  
 time_distributed (TimeDist  (None, 13, 32)               71808     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['time_distributed[0][0]']    
                              (None, 64),                                                   

In [6]:
# we might even need the data generator for the predictions - hold on to this thought for now

In [7]:
# locate the test data

In [8]:
# input features
input_features_loc = 'data/test_input_sub_images'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [9]:
input_contents

['test_data_blk_0103.npy',
 'test_data_blk_0104.npy',
 'test_data_blk_0105.npy',
 'test_data_blk_0106.npy',
 'test_data_blk_0201.npy',
 'test_data_blk_0202.npy',
 'test_data_blk_0205.npy',
 'test_data_blk_0206.npy',
 'test_data_blk_0302.npy',
 'test_data_blk_0303.npy',
 'test_data_blk_0304.npy',
 'test_data_blk_0305.npy',
 'test_data_blk_0306.npy']

In [10]:
# test targets
out_targets_loc = 'data/test_out_targets'
out_contents = os.listdir(out_targets_loc)
out_contents.sort()

In [11]:
out_contents

['test_targets_blk_0103.npy',
 'test_targets_blk_0104.npy',
 'test_targets_blk_0105.npy',
 'test_targets_blk_0106.npy',
 'test_targets_blk_0201.npy',
 'test_targets_blk_0202.npy',
 'test_targets_blk_0205.npy',
 'test_targets_blk_0206.npy',
 'test_targets_blk_0302.npy',
 'test_targets_blk_0303.npy',
 'test_targets_blk_0304.npy',
 'test_targets_blk_0305.npy',
 'test_targets_blk_0306.npy']

In [12]:
trial_test_features = np.load(os.path.join(input_features_loc, input_contents[0]))

In [13]:
trial_test_features.shape

(3072, 13, 100, 100, 3)

In [14]:
trial_test_targets = np.load(os.path.join(out_targets_loc, out_contents[0]))

In [15]:
trial_test_targets.shape

(3072, 7, 32)

In [18]:
%%time
# Get preds in a loop
# now how to proceed? We may need to do a batch predict now using the generator 

batch_size = 32
test_rmse = []
test_mae = []
test_r2 = []
test_pearsonr = []
preds = []
for i in range(len(input_contents)):
    # load the features
    test_features = np.load(os.path.join(input_features_loc, input_contents[i]))
    # load targets
    test_targets = np.load(os.path.join(out_targets_loc, out_contents[i]))
    test_data_gen = DataGenerator(test_features, test_targets, batch_size, shuffle=False)
    # Use the batch predictions to generate the predictions
    test_preds, test_targets_alt = batch_predict(model, test_data_gen, flatten=True, verbose=True)
    preds.append(test_preds)
    print(np.mean(test_targets == test_targets_alt))
    # compute the test scores, I think we need to flatten these before computing the scores - or can use tf, but the answers are going to be the same
    test_preds_flatten = test_preds.flatten()
    test_targets_flatten = test_targets_alt.flatten()
    mae = mean_absolute_error(test_targets_flatten, test_preds_flatten)
    test_mae.append(mae)
    rmse = np.sqrt(mean_squared_error(test_targets_flatten, test_preds_flatten))
    test_rmse.append(rmse)
    rsquare = r2_score(test_targets_flatten, test_preds_flatten)
    test_r2.append(rsquare)
    pearsonr_score = pearsonr(test_targets_flatten, test_preds_flatten)[0]
    test_pearsonr.append(pearsonr_score)

Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
Predictions: 3072
True: 3072
1.0
CPU times: user 3min 7s, sys: 2min 27s, total: 5min 34s
Wall time: 6min 26s


In [19]:
test_rmse

[0.31747225,
 0.33733016,
 0.32803968,
 0.38530892,
 0.33115408,
 0.33046874,
 0.3833809,
 0.36769372,
 0.31380218,
 0.3293048,
 0.3241302,
 0.35331967,
 0.35684893]

In [20]:
test_mae

[0.1478077,
 0.15567987,
 0.14954916,
 0.17246297,
 0.15214655,
 0.1489777,
 0.17234473,
 0.1662072,
 0.14160922,
 0.15481646,
 0.15099059,
 0.16074558,
 0.1618812]

In [21]:
test_r2

[0.6998363846105589,
 0.7643457803414436,
 0.776506259636776,
 0.7460724804232809,
 0.7202018945241722,
 0.7035406125648127,
 0.7557951187434061,
 0.6993896240142365,
 0.7433063749618761,
 0.6689548479563204,
 0.7321527900747793,
 0.7004014576007314,
 0.7160688919291099]

In [22]:
test_pearsonr

[0.839462761676179,
 0.8774650611564724,
 0.8874691152933466,
 0.8793896738958116,
 0.8496702982731692,
 0.8389333598933931,
 0.8851703662613105,
 0.8366626935649869,
 0.8622048473942682,
 0.8249389151331212,
 0.8557162281517302,
 0.8372823024306557,
 0.8467733920404934]

In [24]:
# We have reported the metrics considering the entire test dataset together as well, let's work on that. Pretty sure we will not be able to get it done in a notebook, let's move to a py script.